In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Load data into dataframes
trees = pd.read_csv('../data/2015_Street_Tree_Census_-_Tree_Data.csv')
temps = pd.read_csv('../data/Hyperlocal_Temperature_Monitoring.csv')

###  Convert Datatypes for all columns in `temps` and `trees` to reduce memory usage and enable comparisons

In [3]:
temps.columns

Index(['Sensor.ID', 'AirTemp', 'Day', 'Hour', 'Latitude', 'Longitude', 'Year', 'Install.Type', 'Borough', 'ntacode'], dtype='object')

In [4]:
dtypes = {
    'Sensor.ID': str,
    'AirTemp': float,
    'Latitude': float,
    'Longitude': float,
    'Day': 'datetime64[ns]',
    'Hour': int,
    'Year': int,
    'Install.Type': str,
    'Borough': str,
    'ntacode': str
}

temps = temps.astype(dtypes)

In [5]:
trees.columns

Index(['tree_id', 'block_id', 'created_at', 'tree_dbh', 'stump_diam', 'curb_loc', 'status', 'health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'user_type', 'problems', 'root_stone', 'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other', 'brch_light', 'brch_shoe', 'brch_other', 'address', 'postcode', 'zip_city', 'community board', 'borocode', 'borough', 'cncldist', 'st_assem', 'st_senate', 'nta', 'nta_name', 'boro_ct', 'state', 'latitude', 'longitude', 'x_sp', 'y_sp', 'council district', 'census tract', 'bin', 'bbl'], dtype='object')

In [6]:
dtypes = {
    'created_at': 'datetime64[ns]',
    'tree_id': 'int32',
    'block_id': 'int32',
    'tree_dbh': 'int32',
    'stump_diam': 'int32',
    'curb_loc': 'str',
    'status': 'str',
    'health': 'str',
    'spc_latin': 'str',
    'spc_common': 'str',
    'steward': 'str',
    'guards': 'str',
    'sidewalk': 'str',
    'user_type': 'str',
    'problems': 'str',
    'root_stone': 'str',
    'root_grate': 'str',
    'root_other': 'str',
    'trunk_wire': 'str',
    'trnk_light': 'str',
    'trnk_other': 'str',
    'brch_light': 'str',
    'brch_shoe': 'str',
    'brch_other': 'str',
    'address': 'str',
    'postcode': 'int32',
    'zip_city': 'str',
    'community board': 'int32',
    'borocode': 'int32',
    'borough': 'str',
    'cncldist': 'int32',
    'st_assem': 'int32',
    'st_senate': 'int32',
    'nta': 'str',
    'nta_name': 'str',
    'boro_ct': 'int32',
    'state': 'str',
    'latitude': 'float32',
    'longitude': 'float32',
    'x_sp': 'float32',
    'y_sp': 'float32',
    'census tract': 'float32',
    'bin': 'float32',
    'bbl': 'float32',
}

trees = trees.astype(dtypes)

In [7]:
trees.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,steward,guards,sidewalk,user_type,problems,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,address,postcode,zip_city,community board,borocode,borough,cncldist,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,None,None,NoDamage,TreesCount Staff,None,No,No,No,No,No,No,No,No,No,108-005 70 AVENUE,11375,Forest Hills,406,4,Queens,29,28,16,QN17,Forest Hills,4073900,New York,40.723091,-73.844215,1.027431e+06,202756.765625,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,None,None,Damage,TreesCount Staff,Stones,Yes,No,No,No,No,No,No,No,No,147-074 7 AVENUE,11357,Whitestone,407,4,Queens,19,27,11,QN49,Whitestone,4097300,New York,40.794109,-73.818680,1.034456e+06,228644.843750,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,1or2,None,Damage,Volunteer,None,No,No,No,No,No,No,No,No,No,390 MORGAN AVENUE,11211,Brooklyn,301,3,Brooklyn,34,50,18,BK90,East Williamsburg,3044900,New York,40.717579,-73.936607,1.001823e+06,200716.890625,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,None,None,Damage,Volunteer,Stones,Yes,No,No,No,No,No,No,No,No,1027 GRAND STREET,11211,Brooklyn,301,3,Brooklyn,34,53,18,BK90,East Williamsburg,3044900,New York,40.713539,-73.934456,1.002420e+06,199244.250000,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,None,None,Damage,Volunteer,Stones,Yes,No,No,No,No,No,No,No,No,603 6 STREET,11215,Brooklyn,306,3,Brooklyn,39,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666779,-73.975983,9.909138e+05,182202.421875,39.0,165.0,3025654.0,3.010850e+09


### Filter columns of datasets before merging

In [8]:
# Drop columns that are not relevant to the analysis. This includes:
# - spatial columns (since we are not mapping using GIS)
# - columns that contain redundant information (e.g. borocode encodes same info as borough)
cols_to_drop = [
    'block_id',
    'x_sp',
    'y_sp',
    'zip_city',
    'census tract',
    'borocode',
    'boro_ct',
    'nta_name',
    'cncldist',
    'st_assem',
    'st_senate',
    'community board',
    'council district',
    'census tract',
    'bin',
    'bbl',
    'state'
]

[trees.drop(columns=col, inplace=True) for col in cols_to_drop if col in trees.columns]
trees.head()

,tree_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,steward,guards,sidewalk,user_type,problems,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,address,postcode,borough,nta,latitude,longitude
0,180683,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,None,None,NoDamage,TreesCount Staff,None,No,No,No,No,No,No,No,No,No,108-005 70 AVENUE,11375,Queens,QN17,40.723091,-73.844215
1,200540,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,None,None,Damage,TreesCount Staff,Stones,Yes,No,No,No,No,No,No,No,No,147-074 7 AVENUE,11357,Queens,QN49,40.794109,-73.818680
2,204026,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,1or2,None,Damage,Volunteer,None,No,No,No,No,No,No,No,No,No,390 MORGAN AVENUE,11211,Brooklyn,BK90,40.717579,-73.936607
3,204337,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,None,None,Damage,Volunteer,Stones,Yes,No,No,No,No,No,No,No,No,1027 GRAND STREET,11211,Brooklyn,BK90,40.713539,-73.934456
4,189565,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,None,None,Damage,Volunteer,Stones,Yes,No,No,No,No,No,No,No,No,603 6 STREET,11215,Brooklyn,BK37,40.666779,-73.975983


In [9]:
# Sampling the temp data
# Keeping only hour 0, 6, 12, 18 for each day for each location
temps_sampled = temps[temps['Hour'].isin([0, 6, 12, 18])]

# Dropping columns that are not relevant to the analysis.
cols_to_drop = ['Year']
[temps_sampled.drop(columns=col, inplace=True) for col in cols_to_drop if col in trees.columns]
temps_sampled.head()

,Sensor.ID,AirTemp,Day,Hour,Latitude,Longitude,Year,Install.Type,Borough,ntacode
5,Bk-BR_01,65.965500,2018-06-15,6,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
11,Bk-BR_01,73.428667,2018-06-15,12,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
17,Bk-BR_01,78.066000,2018-06-15,18,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
23,Bk-BR_01,68.077833,2018-06-16,0,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81
29,Bk-BR_01,65.986500,2018-06-16,6,40.666205,-73.91691,2018,Street Tree,Brooklyn,BK81


In [10]:
# Rename column to match trees dataset for joining.
if 'ntacode' in temps_sampled.columns:
    temps_sampled['nta'] = temps_sampled['ntacode']
    temps_sampled.drop(columns=['ntacode'], inplace=True)

/var/folders/wn/dnjty6rj4tv_ssx562thvdpw0000gn/T/ipykernel_3673/3963665355.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temps_sampled['nta'] = temps_sampled['ntacode']
/var/folders/wn/dnjty6rj4tv_ssx562thvdpw0000gn/T/ipykernel_3673/3963665355.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temps_sampled.drop(columns=['ntacode'], inplace=True)


In [11]:
# Trees has more NTA codes than temps, so we will filter temps to only include the NTA codes that are relevant to trees.
relevant_nta_codes = temps_sampled['nta'].unique()
trees_nta_filtered = trees[trees['nta'].isin(relevant_nta_codes)]

In [12]:
# To make the join faster, index on the merging columns and sort the dataframes by the index.
temps_sampled.set_index('nta', inplace=True)
trees_nta_filtered.set_index('nta', inplace=True)

In [2]:
# pd.merge(trees_nta_filtered, temps_sampled, on='nta', how='inner').to_csv('merged_data.csv'
import dask.dataframe as dd

#Construct a dask objects from a pandas objects
dd_temps_sampled = dd.from_pandas(temps_sampled, npartitions=3)
dd_trees_nta_filtered = dd.from_pandas(trees_nta_filtered, npartitions=3)

#merge on key
print(dd.merge(dd_temps_sampled, dd_trees_nta_filtered, on='nta').compute())

# trees_nta_filtered.join(temps_sampled, on='nta', how='inner').to_csv('merged_data.csv')

NameError: name 'temps_sampled' is not defined